In [2]:
from pymongo import MongoClient
import csv
import time
import pandas as pd
import re

In [3]:
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient.ProjectIF29
collection = db['Tweets']

### Generer les attributs du graphe social 

**_id**: id du user\
**friends_count** : nombre de profils suivis\
**followers_count**: nombre de followers ou d'abonnees\
**listed_count**: le nombre de mention publique pour cet utilisateur\
**statuses_count**: le nombre de tweet et retweet emis par le user\
**favourites_count**: nombre de favoris ou nombre de tweets likés depuis la creation du compte \
**accounted_created_time**: jour et heure de la creation du compte\
**tweet_created_time** : jour et heure de la creation du tweet\
**Age du profil**: du premier tweet au dernier tweet\
**agressivite**\
**visibilite**\
**nombre d'url**\
**longueur moyen des tweets**\
**nombre de reponses**\
**retweet_count**: Nombre de fois que ce tweet a ete retweete\
**reply_count**: Nombre de fois que ce tweet a ete repondu\
**favorite_count**: Nombre de fois que ce tweet a ete like

### Chercher le nombre d'utilisateur dans la base

In [3]:
Nombre_utilisateur = []
for user in collection.aggregate([
    {
        "$group": {"_id": "$user.id"}
    }]):
    Nombre_utilisateur.append(user)
len(Nombre_utilisateur)

1843439

In [ ]:
collection.create_index([("user.id", 1)])

In [76]:
from bson.code import Code
debut = time.time()
#-------fonction pour calculer la longueur moyen d'un tweet---------------

longtweet = Code('''
function(tweets) {
        return tweets.length;}
''')

#------------fonction pour calculer le nombre moyen de hshtag et de reference
hsref = Code('''
function(tweets) {
        var total = 0;
        total += tweets.split('#').length - 1;
        return total;
}''')

ref = Code('''
function(tweets) {
        var total = 0;
        total += tweets.split('@').length - 1;
        return total;
}''')
#--------nombre url------- 
NombreUrl = Code('''function (text) {
  // Expression régulière pour trouver les URL
  var urlRegex = /https?:\/\/[^\s/$.?#].[^\s]*/gi;
  
  // Trouver toutes les correspondances avec la regex
  var matches = text.match(urlRegex);
  
  // Renvoyer le nombre total d'URL trouvées
  return matches ? matches.length : 0;
}
''')
dateMin = Code('''
function(created_at){
        var total = Date.parse("Sun May 15 12:34:56 +0000 2023");
        for (var doc of created_at) {
            total = Math.min(total, Date.parse(doc));
        };
        return total;}
''')
dateMax = Code('''
function(created_at){
        return  Date.parse(created_at);}
''')    
  
age = collection.aggregate([
    {
        '$group': {
            '_id': '$user.id',
            'userCreateDate':{'$first': '$user.created_at'},
            'Maxfollowers': {'$max': '$user.followers_count'},
            'Minfollowers': {'$min': '$user.followers_count'},
            'retweet':{'$sum': '$retweet_count'},
            'reply':{'$sum':'$reply_count'},
            'MaxAmi':{'$max':'$user.friends_count'},
            'MinAmi':{'$min': '$user.friends_count'},
            'MaxStatuses':{'$max':'$user.statuses_count'},
            'MinStatuses':{'$min':'$user.statuses_count'},
            'Maxfavourites':{'$max':'$user.favourites_count'},
            'Minfavourites':{'$min':'$user.favourites_count'},
            'Maxmention': {'$max':'$user.listed_count'},
            'Minmention':{'$min':'$user.listed_count'},
            'tweets':{'$push': '$text'},
             'nombre_tweet':{'$count':{}},
            'longTotalTweet':{'$sum':{'$function':{'body':longtweet, 'args':['$text'],'lang':'js'}}},
            'hstag':{'$sum':{'$function':{'body':hsref, 'args':['$text'], 'lang':'js'}}},
            'reference':{'$sum':{'$function':{'body':ref, 'args':['$text'], 'lang':'js'}}},
            'Nombreurl':{'$sum':{'$function':{'body': NombreUrl, 'args':['$text'], 'lang':'js'}}},
            'dateMax':{'$max':{'$function':{'body': dateMax, 'args':['$created_at'], 'lang':'js'}}},
            'dateMin':{'$min':{'$function':{'body': dateMax, 'args':['$created_at'], 'lang':'js'}}},
            'DateUser':{'$min':{'$function':{'body': dateMax, 'args':['$user.created_at'], 'lang':'js'}}}
        }
    }
])
        
entetes=['_id', 'userCreateDate', 'Maxfollowers', 'Minfollowers', 'retweet', 'reply','MaxAmi','MinAmi','MaxStatuses','MinStatuses','Maxfavourites','Minfavourites'
        ,'Maxmention','Minmention', 'nombre_tweet', 'tweets','longTotalTweet', 'hstag', 'reference','Nombreurl', 'dateMax', 'dateMin','DateUser']
with open('User.csv', 'w', newline='', encoding ='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=entetes)
    writer.writeheader()
    writer.writerows(age)
print('---------------------------Temps execution')
print(time.time()-debut)

---------------------------Temps execution
354.49544072151184


In [77]:
df = pd.read_csv('User.csv')
##----------Affichons les 5 premiers elements--------
df.head(5)

,_id,userCreateDate,Maxfollowers,Minfollowers,retweet,reply,MaxAmi,MinAmi,MaxStatuses,MinStatuses,...,Minmention,nombre_tweet,tweets,longTotalTweet,hstag,reference,Nombreurl,dateMax,dateMin,DateUser
0,418,Wed Jul 05 19:52:46 +0000 2006,87904,87890,0,0,2489,2486,66656,66601,...,4535,6,['Re: 4SQ. You can search for “world cup” or ...,680.0,6.0,4.0,5.0,1.529240e+12,1.529060e+12,1.152129e+12
1,730,Fri Jul 14 01:15:50 +0000 2006,621,621,0,0,191,191,12159,12159,...,20,1,"[""RT @Kasparov63: As I've written, it is up to...",139.0,0.0,1.0,0.0,1.529000e+12,1.529000e+12,1.152840e+12
2,761,Fri Jul 14 04:56:28 +0000 2006,4562,4562,0,0,483,483,29724,29724,...,188,1,['Co-sign https://t.co/3fJRCqqaBl'],31.0,0.0,0.0,1.0,1.529097e+12,1.529097e+12,1.152853e+12
3,885,Fri Jul 14 15:12:42 +0000 2006,21491,21491,0,0,1099,1099,13879,13879,...,1883,1,"['If you tape a #WorldCup game to work, that a...",88.0,1.0,0.0,0.0,1.529103e+12,1.529103e+12,1.152890e+12
4,922,Fri Jul 14 17:59:15 +0000 2006,4399,4399,0,0,934,934,11717,11717,...,268,1,['If you’re in SF tomorrow swing by our #GER\u...,140.0,3.0,1.0,1.0,1.529201e+12,1.529201e+12,1.152900e+12


### Age de l'univers- Age des profils
Par l'age du profil, nous entendons la difference entre le premier tweet et le dernier tweet


In [78]:
start = time.time()
debut_univers = df['dateMin'].min()
fin_univers = df['dateMax'].max()
print('-----Temps execution-------')
print(time.time()-start)

-----Temps execution-------
0.029105186462402344


In [79]:
print(debut_univers)

1528942464000.0


In [80]:
print(fin_univers)

1529254872000.0


### Calcul des attributs suivants dans l'univers

**statuses_count**\
**friends_count**\
**favourite_count**\
**listed_count**\
**followers_count**\
**nombre_url**\
**longueur total d'un tweet**\
**frequencetweet**\
**longueumoyen**


In [81]:
start = time.time()
def statuses(df):
    return df['MaxStatuses']-df['MinStatuses']
#----------nombre de tweets dans univers----------------------
df['statuses'] = df.apply(statuses, axis =1)
#------------nombre friend----------------
def friend(df):
    return df['MaxAmi'] - df['MinAmi']
df['friends'] = df.apply(friend, axis = 1)
#------------nombre favourite-------------------------------
def favourite(df):
    return df['Maxfavourites'] - df['Minfavourites']
df['favourites'] = df.apply(favourite, axis =1)
#---------------nombre de mention-----------------------
def mention(df):
    return df['Maxmention']-df['Minmention']
df['mention'] = df.apply(mention, axis =1)
#------------nombre de followers----------------
def followers(df):
    return df['Maxfollowers'] - df['Minfollowers']
df['followers'] = df.apply(followers, axis = 1)
#---------Compiler l'agressivite -----
def agressivite(df):
    return (df['statuses'] + df['friends'])/350
df['agressivite'] = df.apply(agressivite, axis = 1)
print('---------------------------Temps execution')
print(time.time() - start)

---------------------------Temps execution
336.7379858493805


In [115]:
#---------frequence de tweet------------------------
start = time.time()
def frequencetweet(df):
    return df["statuses"]/(df['dateMin'] - fin_univers+1)
df['frenquenTweet'] = df.apply(frequencetweet, axis = 1)
#-------------frenquence friends-------------------------
def frequenceAmi(df):
    return df['friends']/(df['dateMin'] - fin_univers+1)
df['frequenceAmi'] = df.apply(frequenceAmi, axis =1)
#----------longueur moyen de tweet-----------------------
def frequencefavorite(df):
    return df['Maxfavourites']/(df['dateMin'] - fin_univers+1)
df['frequenceFavourite'] = df.apply(frequencefavorite, axis = 1)
#------------frequence mention-------------------------
def frequenceMention(df):
    return df['Maxmention']/(df['dateMin'] - fin_univers+1)
df['frequenceMention'] = df.apply(frequenceMention, axis =1)
#-----------frequence followers------------------------
def frequenceFollowers(df):
    return df['Maxfollowers']/(df['dateMin'] - fin_univers+1)
df['frequenceFollowers'] = df.apply(frequenceFollowers, axis =1)
#-------------longueur moyen de tweet--------------------
def longueurmoyen(df):
    return df['longTotalTweet']/df['nombre_tweet']
df['longMoyen'] = df.apply(longueurmoyen, axis = 1)
print('----------------Temps execution:')
print(start - time.time())

----------------Temps execution:
-385.6585896015167


La determination dans la cellule precendente des frequences dans l'univers pose probleme, car l'age du dernier tweet serait 0 et ses frequence seraient difficiles à calculer à cause du zéro au denominateur. 
Pour celà nous reprenons le calcul en se basant sur la durée l'age du user depuis la création de son compte

#-------------frequence de tweet------------------------
start = time.time()
def frequencetweet(df):
    return df["MaxStatuses"]/(df['dateMax']-df['DateUser'] )
df['frenquenTweet'] = df.apply(frequencetweet, axis = 1)
#-------------frenquence friends-------------------------
def frequenceAmi(df):
    return df['MaxAmi']/(df['dateMax']-df['DateUser'] )
df['frequenceAmi'] = df.apply(frequenceAmi, axis =1)
#-------------frequence favorites-----------------------
def frequencefavorite(df):
    return df['Maxfavourites']/(df['dateMax']-df['DateUser'] )
df['frequenceFavourite'] = df.apply(frequencefavorite, axis = 1)
#------------frequence mention-------------------------
def frequenceMention(df):
    return df['Maxmention']/(df['dateMax']-df['DateUser'] )
df['frequenceMention'] = df.apply(frequenceMention, axis =1)
#-----------frequence followers------------------------
def frequenceFollowers(df):
    return df['Maxfollowers']/(df['dateMax']-df['DateUser'] )
df['frequenceFollowers'] = df.apply(frequenceFollowers, axis =1)
#-------------longueur moyen de tweet--------------------
def longueurmoyen(df):
    return df['longTotalTweet']/df['nombre_tweet']
df['longMoyen'] = df.apply(longueurmoyen, axis = 1)
#-------------
print('----------------Temps execution:')
print(time.time()-start)

In [116]:
start = time.time()
def hstagmoyenTweet(df):
    return df['hstag']/df['nombre_tweet']
df['hstagmoyenTweet'] = df.apply(hstagmoyenTweet, axis = 1)
#--------------------------------------------------------------
def refermoyenTweet(df):
    return df['reference']/df['nombre_tweet']
df['refermoyenTweet'] = df.apply(refermoyenTweet, axis = 1)
print('----------------------------')
print(time.time()-start)

----------------------------
117.13260269165039


In [117]:
df.shape

(1843439, 38)

### Calcul de la visibilite

### Calcul des nouveaux champs

Dans le but de déterminer l'age d'un utilisateur dans l'Univers, nous allons chercher la date du commencement de l'univers
(date du premier twitter recueillie dans l'univers) et la date de fin de l'univers(la date du dernier tweet)\
Pour ce faire nous allons d'abord :


#### - Convertir les dates et les mettre dans un nouveau champs Tmin(date premier tweet en seconde) et Tmax(date dernier tweet en seconde) et sortir le min et le max

**longueur moyen tweet**\
**frequence_statuses**\
**frequence_friends**

In [87]:
df.head(5)

,_id,userCreateDate,Maxfollowers,Minfollowers,retweet,reply,MaxAmi,MinAmi,MaxStatuses,MinStatuses,...,favourites,mention,followers,agressivite,frenquenTweet,frequenceAmi,frequenceFavourite,frequenceMention,frequenceFollowers,longMoyen
0,418,Wed Jul 05 19:52:46 +0000 2006,87904,87890,0,0,2489,2486,66656,66601,...,38,2,14,0.165714,1.767543e-07,6.600176e-09,3.585946e-08,1.203094e-08,2.330984e-07,113.333333
1,730,Fri Jul 14 01:15:50 +0000 2006,621,621,0,0,191,191,12159,12159,...,0,0,0,0.000000,3.232402e-08,5.077628e-10,8.411317e-09,5.316888e-11,1.650894e-09,139.000000
2,761,Fri Jul 14 04:56:28 +0000 2006,4562,4562,0,0,483,483,29724,29724,...,0,0,0,0.000000,7.900197e-08,1.283742e-09,1.229363e-07,4.996761e-10,1.212512e-08,31.000000
3,885,Fri Jul 14 15:12:42 +0000 2006,21491,21491,0,0,1099,1099,13879,13879,...,0,0,0,0.000000,3.689134e-08,2.921218e-09,1.619030e-08,5.005144e-09,5.712456e-08,88.000000
4,922,Fri Jul 14 17:59:15 +0000 2006,4399,4399,0,0,934,934,11717,11717,...,0,0,0,0.000000,3.113728e-08,2.482054e-09,3.201158e-08,7.121953e-10,1.169010e-08,140.000000


In [126]:
df.iloc[191164]

_id                                                            99943864
userCreateDate                           Mon Dec 28 13:08:48 +0000 2009
Maxfollowers                                                     906473
Minfollowers                                                     906343
retweet                                                               0
reply                                                                 0
MaxAmi                                                           426537
MinAmi                                                                0
MaxStatuses                                                      333283
MinStatuses                                                      329811
Maxfavourites                                                    118752
Minfavourites                                                    117380
Maxmention                                                          753
Minmention                                                      

In [90]:
df.describe()

,_id,Maxfollowers,Minfollowers,retweet,reply,MaxAmi,MinAmi,MaxStatuses,MinStatuses,Maxfavourites,...,favourites,mention,followers,agressivite,frenquenTweet,frequenceAmi,frequenceFavourite,frequenceMention,frequenceFollowers,longMoyen
count,1.843439e+06,1.843439e+06,1.843439e+06,1843439.0,1843439.0,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,...,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06,1.843439e+06
mean,2.276477e+17,3.602963e+03,3.598628e+03,0.0,0.0,8.718743e+02,8.680995e+02,1.768635e+04,1.766600e+04,8.762232e+03,...,1.776090e+01,2.546306e-01,4.335208e+00,6.892047e-02,1.957751e-07,9.536412e-08,1.119202e-07,1.420760e-10,2.504874e-08,1.089201e+02
std,3.886710e+17,1.185683e+05,1.185233e+05,0.0,0.0,4.897467e+03,4.870705e+03,4.049905e+04,4.046705e+04,2.204889e+04,...,2.142029e+02,4.532625e+01,9.199455e+02,1.046694e+00,1.462293e-06,4.437179e-06,8.756703e-07,3.991953e-09,4.724388e-07,3.179056e+01
min,4.180000e+02,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.113030e-12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00
25%,3.033891e+08,1.040000e+02,1.030000e+02,0.0,0.0,1.680000e+02,1.670000e+02,1.108000e+03,1.101000e+03,4.450000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.336703e-08,1.414087e-09,4.227423e-09,0.000000e+00,9.962480e-10,8.900000e+01
50%,1.460774e+09,2.980000e+02,2.970000e+02,0.0,0.0,3.580000e+02,3.560000e+02,5.210000e+03,5.199000e+03,2.168000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.601580e-08,2.982235e-09,1.977980e-08,1.353950e-11,2.454339e-09,1.140000e+02
75%,7.125350e+17,7.540000e+02,7.520000e+02,0.0,0.0,7.590000e+02,7.570000e+02,1.789600e+04,1.787400e+04,7.931000e+03,...,1.000000e+00,0.000000e+00,0.000000e+00,1.428571e-02,1.382681e-07,7.344963e-09,7.610738e-08,5.645906e-11,6.618102e-09,1.400000e+02
max,1.008393e+18,4.220341e+07,4.219805e+07,0.0,0.0,1.569366e+06,1.569366e+06,8.910230e+06,8.898257e+06,1.718515e+06,...,1.067820e+05,5.194800e+04,9.762130e+05,1.228597e+03,3.643411e-04,1.862069e-03,3.196881e-04,2.865330e-06,1.575309e-04,4.160000e+02


### Determination des couts du hastag(#) et du tag(@)
Pour calculer le nombre les couts du hashtag(#) et de tag(@). Nous allons baser sur le fait que ces actions attirent la visibilité; c'est à dire le nombre de **friends**,**followers**,**mention**, **favourites** et que la moyenne hashtag provoque la somme des moyennes de ces elements. Par regle de 3 nous estimons que la valeurs ou le cout est le nombre moyen de ces elements que produirait 1 hashtag ou 1 reference

In [112]:
df['hstag'].mean()/((df['friends']/df['nombre_tweet']).mean()+(df['favourites']/df['nombre_tweet']).mean()+
(df['mention']/df['nombre_tweet']).mean()+ (df['followers']/df['nombre_tweet']).mean())
                             

0.7884605111452351

In [101]:
df['refermoyenTweet'].mean()/(df['friends'].mean()+df['favourites'].mean()+df['mention'].mean()+df['followers'].mean())

0.03513033595729966

In [118]:
start = time.time()
def visibilite(df):
    return (df['hstagmoyenTweet']*11.6 + df['reference']*11.4)/140
df['visibilite'] = df.apply(visibilite, axis=1)
print(time.time()-start)

60.68363857269287


In [120]:
df.to_csv('kmeans.csv', sep=',')
print('fin')

fin


### Creer la base de donnees Mongodb des Users

In [121]:
##Transformer la dataframe en dictionnaire
start = time.time()
df_dict = df.to_dict('records')
print('-------------------------------Temps execution')
print(time.time() - start)

-------------------------------Temps execution
55.30426502227783


In [125]:
##Inserer le dictionnaire 
user = db['utilisateur']
start = time.time()
user.insert_many(df_dict)
print('-----------------------------Temps execution')
print(time.time() - start)

-----------------------------Temps execution
276.8498225212097


In [39]:
df.head(5)

,_id,first_tweet,last_tweet,MaxAmi,MinAmi,MaxStatuses,MinStatuses,Maxfavourites,Minfavourites,Maxmention,...,tweets,longTotalTweet,hstag,Nombreurl,T1,T2,statuses,friends,favourites,mention
0,418,Sat Jun 16 04:01:01 +0000 2018,Fri Jun 15 10:46:07 +0000 2018,2489,2486,66656,66601,13523,13485,4537,...,['Re: 4SQ. You can search for “world cup” or ...,680.0,10.0,5.0,1.529114e+09,1.529114e+09,55,3,38,2
1,730,Thu Jun 14 18:06:49 +0000 2018,Thu Jun 14 18:06:49 +0000 2018,191,191,12159,12159,3164,3164,20,...,"[""RT @Kasparov63: As I've written, it is up to...",139.0,1.0,0.0,1.528992e+09,1.528992e+09,0,0,0,0
2,761,Fri Jun 15 21:05:55 +0000 2018,Fri Jun 15 21:05:55 +0000 2018,483,483,29724,29724,46254,46254,188,...,['Co-sign https://t.co/3fJRCqqaBl'],31.0,0.0,1.0,1.529090e+09,1.529090e+09,0,0,0,0
3,885,Fri Jun 15 22:48:44 +0000 2018,Fri Jun 15 22:48:44 +0000 2018,1099,1099,13879,13879,6091,6091,1883,...,"['If you tape a #WorldCup game to work, that a...",88.0,1.0,0.0,1.529096e+09,1.529096e+09,0,0,0,0
4,922,Sun Jun 17 02:07:11 +0000 2018,Sun Jun 17 02:07:11 +0000 2018,934,934,11717,11717,12046,12046,268,...,['If you’re in SF tomorrow swing by our #GER\u...,140.0,4.0,1.0,1.529194e+09,1.529194e+09,0,0,0,0
